In [0]:
from google.colab import files
uploaded = files.upload()
!ls
!mkdir /root/.kaggle
!mv kaggle.json /root/.kaggle/
!kaggle competitions download -c Kannada-MNIST
!unzip test.csv.zip
!unzip train.csv.zip
!unzip Dig-MNIST.csv.zip
!ls

Saving kaggle.json to kaggle.json
kaggle.json  sample_data
  0% 0.00/33.1k [00:00<?, ?B/s]
100% 33.1k/33.1k [00:00<00:00, 12.2MB/s]
  0% 0.00/1.90M [00:00<?, ?B/s]
100% 1.90M/1.90M [00:00<00:00, 62.7MB/s]
  0% 0.00/818k [00:00<?, ?B/s]
100% 818k/818k [00:00<00:00, 114MB/s]
 52% 5.00M/9.71M [00:00<00:00, 21.8MB/s]
100% 9.71M/9.71M [00:00<00:00, 32.2MB/s]
Archive:  test.csv.zip
  inflating: test.csv                
Archive:  train.csv.zip
  inflating: train.csv               
Archive:  Dig-MNIST.csv.zip
  inflating: Dig-MNIST.csv           
Dig-MNIST.csv	   sample_data		  test.csv	train.csv
Dig-MNIST.csv.zip  sample_submission.csv  test.csv.zip	train.csv.zip


In [0]:
%tensorflow_version 1.x
import tensorflow as tf

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
tf.keras.backend.set_session(tf.Session(config=config))

from keras import backend as K

# from keras.applications.vgg16 import VGG16
# from keras.applications.vgg16 import preprocess_input

from keras.models import Sequential
from keras.layers import Conv2D, Dropout, MaxPooling2D, Flatten, Dense

from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD, Adam, RMSprop
from keras.callbacks import LearningRateScheduler
from keras.utils.np_utils import to_categorical
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn import metrics
# from sklearn.metrics import accuracy_score, classification_report
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

Using TensorFlow backend.


In [0]:
from keras.layers import LeakyReLU
from keras.layers.normalization import BatchNormalization

In [0]:
train = pd.read_csv('train.csv', header=0)
test = pd.read_csv('test.csv', header=0)
sample_sub = pd.read_csv('sample_submission.csv', header=0)
dig  = pd.read_csv('Dig-MNIST.csv', header = 0)

In [0]:
print("Train set shape = " +str(train.shape))
print("Test set shape = " +str(test.shape))
print("Sub set shape = " +str(sample_sub.shape))
print("Dig set shape = " +str(dig.shape))

Train set shape = (60000, 785)
Test set shape = (5000, 785)
Sub set shape = (5000, 2)
Dig set shape = (10240, 785)


In [0]:
x_train = train.values[:,1:].reshape(train.shape[0], 28, 28, 1)
x_train = x_train.astype('float32')
x_train = x_train / 255.0
y_train = train.values[:,0]
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size = 0.1, random_state=2019)
x_test = test.values[:,1:].reshape(test.shape[0], 28, 28, 1)
x_test = x_test.astype('float32')
x_test = x_test / 255.0

In [0]:
print(x_train.shape)

(54000, 28, 28, 1)


In [0]:
lb = preprocessing.LabelBinarizer()
y_train = lb.fit_transform(y_train)
y_valid = lb.fit_transform(y_valid)

In [0]:
modelo = Sequential()

modelo.add(Conv2D(64,  (3,3), padding='same', input_shape=(28, 28, 1)))
modelo.add(BatchNormalization(momentum=0.1, epsilon=1e-5, gamma_initializer="uniform"))
modelo.add(LeakyReLU(alpha=0.1))

modelo.add(MaxPooling2D(2, 2))
modelo.add(Dropout(0.2))

modelo.add(Conv2D(128, (3,3), padding='same'))
modelo.add(BatchNormalization(momentum=0.2, epsilon=1e-5, gamma_initializer="uniform"))
modelo.add(LeakyReLU(alpha=0.1))
modelo.add(Conv2D(128, (3,3), padding='same'))
modelo.add(BatchNormalization(momentum=0.1, epsilon=1e-5, gamma_initializer="uniform"))
modelo.add(LeakyReLU(alpha=0.1))

modelo.add(MaxPooling2D(2,2))
modelo.add(Dropout(0.2))

modelo.add(Conv2D(256, (3,3), padding='same'))
modelo.add(BatchNormalization(momentum=0.2, epsilon=1e-5, gamma_initializer="uniform"))
modelo.add(LeakyReLU(alpha=0.1))
modelo.add(Conv2D(256, (3,3), padding='same'))
modelo.add(BatchNormalization(momentum=0.1, epsilon=1e-5, gamma_initializer="uniform"))
modelo.add(LeakyReLU(alpha=0.1))

modelo.add(MaxPooling2D(2,2))
modelo.add(Dropout(0.2))

modelo.add(Flatten())
modelo.add(Dense(256))
modelo.add(LeakyReLU(alpha=0.1))

modelo.add(BatchNormalization())
modelo.add(Dense(10, activation='softmax'))












Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
initial_learningrate=2e-3
def lr_decay(epoch):#lrv
    return initial_learningrate * 0.99 ** epoch
modelo.compile(loss= 'categorical_crossentropy', optimizer= RMSprop(lr=initial_learningrate) , metrics=['accuracy'])
modelo.summary()



Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 64)        640       
_________________________________________________________________
batch_normalization_1 (Batch (None, 28, 28, 64)        256       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 28, 28, 64)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 128)       73856     
_________________________________________________________________
batch_normalization_2 (Batch (None, 14, 14, 128)    

In [0]:
epochs=50
batch_size=200
history = modelo.fit(x_train, y_train,
          epochs=epochs,
          batch_size= batch_size,
          callbacks=[LearningRateScheduler(lr_decay)],
          validation_data=(x_valid, y_valid))
